# EMA Scanner

In [17]:
from tabulate import tabulate
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import datetime

# Define the tickers and time rangenifty50_symbols
tickers = ["HDFCBANK.NS", "TATASTEEL.NS", "WIPRO.NS", "ICICIBANK.NS", "RELIANCE.NS"]


start_time = datetime.time(9, 30)  # 09:30 AM
end_time = datetime.time(15, 0)  # 03:00 PM

def fetch_and_calculate_ema(ticker):
    # Fetching 5min interval data for the current day, suppress progress bar
    data = yf.download(ticker, interval='5m', period='1mo', progress=False)
    
    # Calculating 10 and 20 EMA
    
    data['EMA10'] = ta.ema(data['Close'], length=25)
    data['EMA20'] = ta.ema(data['Close'], length=44)
    
    return data

def check_crossover(data, ticker):
    crossover_events = pd.DataFrame()
    for i in range(1, len(data)):
        current_time = data.index[i].time()
        if start_time <= current_time <= end_time:
            # Check for positive crossover
            if (data['EMA10'].iloc[i] > data['EMA20'].iloc[i]) and (data['EMA10'].iloc[i-1] < data['EMA20'].iloc[i-1]):
                event = pd.DataFrame([{
                    "DateTime": data.index[i],
                    "Ticker": ticker,
                    "Crossover Type": "Positive EMA Crossover",
                    "Close": data['Close'].iloc[i],
                    "EMA10": data['EMA10'].iloc[i],
                    "EMA20": data['EMA20'].iloc[i]
                }])
                crossover_events = pd.concat([crossover_events, event])
            
            # Check for negative crossover
            elif (data['EMA10'].iloc[i] < data['EMA20'].iloc[i]) and (data['EMA10'].iloc[i-1] > data['EMA20'].iloc[i-1]):
                event = pd.DataFrame([{
                    "DateTime": data.index[i],
                    "Ticker": ticker,
                    "Crossover Type": "Negative EMA Crossover",
                    "Close": data['Close'].iloc[i],
                    "EMA10": data['EMA10'].iloc[i],
                    "EMA20": data['EMA20'].iloc[i]
                }])
                crossover_events = pd.concat([crossover_events, event])
                
    return crossover_events



# ANSI color codes
RED = '\033[31m'
GREEN = '\033[32m'
RESET = '\033[0m'


def scan_tickers():
    all_crossovers = pd.DataFrame()
    for ticker in tickers:
        data = fetch_and_calculate_ema(ticker)
        crossover_events = check_crossover(data, ticker)
        all_crossovers = pd.concat([all_crossovers, crossover_events])
    
    if not all_crossovers.empty:
        # Convert 'DateTime' to a datetime object if it's not already
        all_crossovers['DateTime'] = pd.to_datetime(all_crossovers['DateTime'])
        
        # Round off the values to two decimal places
        all_crossovers['Close'] = all_crossovers['Close'].round(2)
        all_crossovers['EMA10'] = all_crossovers['EMA10'].round(2)
        all_crossovers['EMA20'] = all_crossovers['EMA20'].round(2)
        
        # Sort by DateTime
        all_crossovers.sort_values(by='DateTime', inplace=True)

        # Apply color coding to the 'Crossover Type'
        all_crossovers['Crossover Type'] = all_crossovers['Crossover Type'].apply(
            lambda x: f"{GREEN}{x}{RESET}" if "Positive" in x else f"{RED}{x}{RESET}"
        )

        # Tabulate and print
        tabulated_data = tabulate(all_crossovers, headers='keys', tablefmt='fancy_grid', showindex=False)
        print(tabulated_data)
        return True

# Run the scan
scan_tickers()

╒═══════════════════════════╤══════════════╤════════════════════════╤═════════╤═════════╤═════════╕
│ DateTime                  │ Ticker       │ Crossover Type         │   Close │   EMA10 │   EMA20 │
╞═══════════════════════════╪══════════════╪════════════════════════╪═════════╪═════════╪═════════╡
│ 2024-01-03 12:55:00+05:30 │ WIPRO.NS     │ Negative EMA Crossover │  458.05 │  458.73 │  458.74 │
├───────────────────────────┼──────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-04 10:05:00+05:30 │ HDFCBANK.NS  │ Positive EMA Crossover │ 1685.35 │ 1677.61 │ 1677.4  │
├───────────────────────────┼──────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-04 10:55:00+05:30 │ RELIANCE.NS  │ Positive EMA Crossover │ 2602.6  │ 2599.92 │ 2599.89 │
├───────────────────────────┼──────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-04 11:10:00+05:30 │ RELIANCE.NS  │ Negative EMA Crossover │ 2597.45 │ 2599.8  │ 2599.83 │


True

In [25]:
from tabulate import tabulate
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import datetime

# Define the tickers and time range
tickers = ['AAVAS.NS', 'AEGISCHEM.NS', 'AFFLE.NS', 'ARE&M.NS', 'AMBER.NS', 'ANGELONE.NS', 'ANURAS.NS', 'APARINDS.NS', 'BLS.NS', 'BSE.NS', 'BALAMINES.NS', 'BALRAMCHIN.NS', 'BIKAJI.NS', 'BIRLACORPN.NS', 'BSOFT.NS', 'CESC.NS', 'CIEINDIA.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYTEX.NS', 'CHAMBLFERT.NS', 'CUB.NS', 'COCHINSHIP.NS', 'CAMS.NS', 'CREDITACC.NS', 'CYIENT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'EASEMYTRIP.NS', 'ELGIEQUIP.NS', 'EQUITASBNK.NS', 'EXIDEIND.NS', 'FINCABLES.NS', 'FSL.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GNFC.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HINDCOPPER.NS', 'HUDCO.NS', 'IDBI.NS', 'IDFC.NS', 'IIFL.NS', 'IRB.NS', 'INDIAMART.NS', 'IEX.NS', 'IOB.NS', 'INTELLECT.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JKLAKSHMI.NS', 'JUBLINGREA.NS', 'JYOTHYLAB.NS', 'KEI.NS', 'KRBL.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KEC.NS', 'LATENTVIEW.NS', 'LXCHEM.NS', 'MGL.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MEDPLUS.NS', 'METROPOLIS.NS', 'MCX.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NLCINDIA.NS', 'NSLNISP.NS', 'NATIONALUM.NS', 'PNBHOUSING.NS', 'PVRINOX.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'RBLBANK.NS', 'RITES.NS', 'RADICO.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'ROUTE.NS', 'SJVN.NS', 'RENUKA.NS', 'SONATSOFTW.NS', 'SUZLON.NS', 'TANLA.NS', 'TTML.NS', 'TEJASNET.NS', 'TRITURBINE.NS', 'UCOBANK.NS', 'UTIAMC.NS', 'VIPIND.NS', 'WELSPUNLIV.NS', 'ZENSARTECH.NS']

start_time = datetime.time(9, 30)  # 09:30 AM
end_time = datetime.time(15, 0)  # 03:00 PM

def fetch_and_calculate_ema(ticker):
    # Fetching 1d interval data for the last 1 month, suppress progress bar
    data = yf.download(ticker, interval='1h', period='1mo', progress=False)
    
    # Calculating 25 and 44 EMA
    data['EMA10'] = ta.ema(data['Close'], length=25)
    data['EMA20'] = ta.ema(data['Close'], length=44)
    
    return data

def check_crossover(data, ticker):
    crossover_events = pd.DataFrame()
    for i in range(1, len(data)):
        current_time = data.index[i].time()
        if start_time <= current_time <= end_time:
            # Check for positive crossover
            if (data['EMA10'].iloc[i] > data['EMA20'].iloc[i]) and (data['EMA10'].iloc[i-1] <= data['EMA20'].iloc[i-1]):
                event = pd.DataFrame([{
                    "DateTime": data.index[i],
                    "Ticker": ticker,
                    "Crossover Type": "Positive EMA Crossover",
                    "Close": data['Close'].iloc[i],
                    "EMA10": data['EMA10'].iloc[i],
                    "EMA20": data['EMA20'].iloc[i]
                }])
                crossover_events = pd.concat([crossover_events, event])
            
            # Check for negative crossover
            elif (data['EMA10'].iloc[i] < data['EMA20'].iloc[i]) and (data['EMA10'].iloc[i-1] >= data['EMA20'].iloc[i-1]):
                event = pd.DataFrame([{
                    "DateTime": data.index[i],
                    "Ticker": ticker,
                    "Crossover Type": "Negative EMA Crossover",
                    "Close": data['Close'].iloc[i],
                    "EMA10": data['EMA10'].iloc[i],
                    "EMA20": data['EMA20'].iloc[i]
                }])
                crossover_events = pd.concat([crossover_events, event])
                
    return crossover_events

def scan_tickers():
    all_crossovers = pd.DataFrame()
    for ticker in tickers:
        data = fetch_and_calculate_ema(ticker)
        crossover_events = check_crossover(data, ticker)
        all_crossovers = pd.concat([all_crossovers, crossover_events])
    
    if not all_crossovers.empty:
        # Convert 'DateTime' to a datetime object if it's not already
        all_crossovers['DateTime'] = pd.to_datetime(all_crossovers['DateTime'])
        
        # Round off the values to two decimal places
        all_crossovers['Close'] = all_crossovers['Close'].round(2)
        all_crossovers['EMA10'] = all_crossovers['EMA10'].round(2)
        all_crossovers['EMA20'] = all_crossovers['EMA20'].round(2)
        
        # Sort by DateTime
        all_crossovers.sort_values(by='DateTime', inplace=True)

        # Apply color coding to the 'Crossover Type'
        all_crossovers['Crossover Type'] = all_crossovers['Crossover Type'].apply(
            lambda x: f"{GREEN}{x}{RESET}" if "Positive" in x else f"{RED}{x}{RESET}"
        )

        # Tabulate and print
        tabulated_data = tabulate(all_crossovers, headers='keys', tablefmt='fancy_grid', showindex=False)
        print(tabulated_data)
        return True

# Run the scan
scan_tickers()


╒═══════════════════════════╤═══════════════╤════════════════════════╤═════════╤═════════╤═════════╕
│ DateTime                  │ Ticker        │ Crossover Type         │   Close │   EMA10 │   EMA20 │
╞═══════════════════════════╪═══════════════╪════════════════════════╪═════════╪═════════╪═════════╡
│ 2024-01-11 13:15:00+05:30 │ NLCINDIA.NS   │ Positive EMA Crossover │  233.1  │  227.81 │  227.75 │
├───────────────────────────┼───────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-11 13:15:00+05:30 │ IEX.NS        │ Positive EMA Crossover │  165.15 │  164.04 │  164.02 │
├───────────────────────────┼───────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-11 13:15:00+05:30 │ JYOTHYLAB.NS  │ Negative EMA Crossover │  498    │  503.23 │  503.33 │
├───────────────────────────┼───────────────┼────────────────────────┼─────────┼─────────┼─────────┤
│ 2024-01-11 14:15:00+05:30 │ FINCABLES.NS  │ Positive EMA Crossover │ 1046    │ 1037.83 │ 

True